In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import datetime
import backtrader as bt

import talib 
from futu import *
import datetime
import os
import empyrical as ey



from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

In [2]:
from collections import OrderedDict

In [3]:
import empyrical as ey


In [4]:
#用 backtrader_plotting 的 bokeh 绘图

from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo

Loading BokehJS ...

In [5]:
###用dash 绘图

import jupyterlab_dash
import dash
import dash_html_components as html
import dash_core_components as dcc

In [6]:
### 聚宽数据

from jqdatasdk import *
auth('18688709107','chan123456')


## 聚宽北上、 南下 交易量数据
table = finance.STK_ML_QUOTA
df_hk  = finance.run_query(query(
        table.day, table.quota_daily, table.quota_daily_balance
    ).filter(
        table.link_id.in_(['310003', '310004']), table.day<='2020-09-20'  #沪股通、深股通
    ).order_by(table.day))
df_cn  = finance.run_query(query(
        table.day, table.quota_daily, table.quota_daily_balance
    ).filter(
        table.link_id.in_(['310001', '310002']), table.day<='2020-09-20'  #沪股通、深股通
    ).order_by(table.day))


提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改
auth success 


In [7]:
start_date = '2017-8-01'
end_date ='2020-09-12'
stock_list =['HK.00700']

In [8]:
# 从富途获取股票数据

def get_stockdata(stock_list,start_date,end_date):
    df={}
    for stock in stock_list:
        
        quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
        ret, stock_data, page_req_key = quote_ctx.request_history_kline(stock, start=start_date, end=end_date)  
        quote_ctx.close() 
        
        
        stock_data['date']=pd.to_datetime(stock_data['time_key'])
        stock_data=stock_data.set_index('date',drop=True)
        stock_data=stock_data.drop(columns=['time_key'])
        
        stock_data['openinterest']=0
        df.update({stock:stock_data})
    return df
    
   
    

In [9]:
df_stock=get_stockdata(stock_list,start_date,end_date)

2020-09-24 09:00:32,932 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2020-09-24 09:00:32,940 [open_context_base.py] on_connected:344: Connected : conn_id=1; 
2020-09-24 09:00:32,947 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=1; info={'server_version': 216, 'login_user_id': 245840, 'conn_id': 6714926491775733704, 'conn_key': '54D37A549B91C8EB', 'conn_iv': '7960B8D7C201EC79', 'keep_alive_interval': 10, 'is_encrypt': False};
2020-09-24 09:00:33,161 [network_manager.py] work:366: Close: conn_id=1


In [10]:
df_stock

{'HK.00700':                 code    open   close    high     low  pe_ratio  turnover_rate  \
 date                                                                            
 2017-08-01  HK.00700  305.92  311.32  314.32  305.92    64.545        0.00219   
 2017-08-02  HK.00700  311.92  309.52  314.72  308.72    64.175        0.00195   
 2017-08-03  HK.00700  307.72  305.52  309.52  304.92    63.354        0.00247   
 2017-08-04  HK.00700  308.12  308.32  312.32  306.72    63.929        0.00181   
 2017-08-07  HK.00700  312.32  317.52  318.32  311.92    65.818        0.00224   
 ...              ...     ...     ...     ...     ...       ...            ...   
 2020-09-07  HK.00700  516.00  511.00  524.50  510.00    47.539        0.00188   
 2020-09-08  HK.00700  517.00  508.50  520.00  502.00    47.306        0.00193   
 2020-09-09  HK.00700  502.00  504.00  508.50  501.50    46.888        0.00185   
 2020-09-10  HK.00700  514.00  506.00  518.50  506.00    47.074        0.00143   
 202

In [11]:
#从文件导入数据

df_NX=pd.read_csv('南下.csv',index_col=0)

In [15]:
##交易函数

class TXStrategy(bt.Strategy):
    # 设置简单均线周期，以备后面调用

    params = (
        ('ssa_window', 15),
        ('myperiod', 60),
    )


    def log(self, txt, dt=None):
        # 日记记录输出
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 初始化数据参数
        
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.signal_1='no'
        self.buylist =[]
        self.boll_window = 120
        self.lastlist =[]
        self.GGT = df_NX
        self.stocklist=stock_list

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 如果有订单提交或者已经接受的订单，返回退出
            return
            # 主要是检查有没有成交的订单，如果有则日志记录输出价格，金额，手续费。注意，如果资金不足是不会成交订单的
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED ,Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:  
            return
        self.log(' OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
        
    def get_boll(self,end_date):
        """
        获取北向资金布林带    
        
        净买入额在布林线下轨以下时清仓
        净买入额在布林线上轨以上时执行调仓
        """
        
        stdev_n = 2
        money_df = self.GGT
        if len(money_df[money_df.day==end_date]) ==0:
            return
        
        idx = money_df[money_df.day==end_date].index[-1]
        money_df=money_df.loc[:idx].copy()
        
        money_df['net_amount'] = money_df['quota_daily'] - money_df['quota_daily_balance'] #每日额度-每日剩余额度=净买入额
        # 分组求和
        money_df = money_df.groupby('day')[['net_amount']].sum().iloc[-self.params.myperiod:] #过去g.window天求和
        mid = money_df['net_amount'].mean()
        stdev = money_df['net_amount'].std()
        upper = mid + stdev_n * stdev
        lower = mid - stdev_n * stdev
        mf = money_df['net_amount'].iloc[-1]
        
        if mf > int(upper):
            #print('----------------------buy------------')
            return 'buy'
                        
        # 净买入额在布林线下轨以下时清仓
        elif  mf <= lower:
            #print('----------------------sell------------')
            return 'sell'
        else:
            return 'no signal'
             
    def next(self):
        date = self.datas[0].datetime.date(0).isoformat()
        signal = self.get_boll(date)
        daily_value = self.broker.get_value()
        if signal == 'sell' :
            for data in self.datas:
                self.close(data)              
            
            self.lastlist=[]
        
        elif signal == 'buy':    
            buycash = daily_value/1
            for data in self.datas:
                    self.order_target_value(data,buycash*0.9)
                    self.lastlist.append(str(data._name))                 
        else:
            pass
        #self.log(' %s  :%.2f '%(self.datas[0]._name,self.datas[0].close[0]))
        #self.log('户头总值: %.2f' % daily_value)
                    
            #for buy_stock in self.buylist:
                #self.order_target_size(target=1)
    
    def settingCerebro(self):
        # 添加回撤观察器
        self.__cerebro.addobserver(bt.observers.DrawDown)
        # 设置手续费
        self.__cerebro.broker.setcommission(commission=self.__commission)
        # 设置初始资金为0.01
        self.__cerebro.broker.setcash(self.__initcash)
        # 添加分析对象
        self.__cerebro.addanalyzer(btay.SharpeRatio, _name = "sharpe", riskfreerate = 0.02)
        self.__cerebro.addanalyzer(btay.AnnualReturn, _name = "AR")
        self.__cerebro.addanalyzer(btay.DrawDown, _name = "DD")
        self.__cerebro.addanalyzer(btay.Returns, _name = "RE")
        self.__cerebro.addanalyzer(btay.TradeAnalyzer, _name = "TA")                        

    # 获取回测指标
    def getResult(self):
        return self.__backtestResult


In [16]:
cerebro = bt.Cerebro()

cerebro.addstrategy(TXStrategy)

#data = bt.feeds.PandasData(dataname=TX_data,name='HK.00700',fromdate=datetime.datetime(2019, 12, 1),todate=datetime.datetime(2020, 8, 31))
#data2 = bt.feeds.PandasData(dataname=df_stock['HK.09988'],name='HK.09988',fromdate=datetime.datetime(2019, 12, 1),todate=datetime.datetime(2020, 8, 31))
    
for s in stock_list:
    feed = bt.feeds.PandasData(dataname =df_stock[s] ,fromdate=datetime.datetime(2019, 12, 1),todate=datetime.datetime(2020,9, 12))
    cerebro.adddata(feed, name = s)

cerebro.broker.setcash(1000000.0)
    # 设置每笔交易交易的股票数量
cerebro.addsizer(bt.sizers.FixedSize, stake=100)
    # 设置手续费
cerebro.broker.setcommission(commission=0.002)

    # 输出初始资金
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # 运行策略

    # 输出结果
    
#cerebro.optstrategy(TXStrategy,myperiod=range(100, 140,10))    


cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
cerebro.addanalyzer(bt.analyzers.Calmar, _name='_Calmar')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
# cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name='_TimeDrawDown')
cerebro.addanalyzer(bt.analyzers.GrossLeverage, _name='_GrossLeverage')
cerebro.addanalyzer(bt.analyzers.PositionsValue, _name='_PositionsValue')
cerebro.addanalyzer(bt.analyzers.LogReturnsRolling, _name='_LogReturnsRolling')
cerebro.addanalyzer(bt.analyzers.PeriodStats, _name='_PeriodStats')
cerebro.addanalyzer(bt.analyzers.Returns, _name='_Returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio')
# cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='_SharpeRatio_A')
cerebro.addanalyzer(bt.analyzers.SQN, _name='_SQN')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='_TradeAnalyzer')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='_Transactions')
cerebro.addanalyzer(bt.analyzers.VWR, _name='_VWR')
#cerebro.addanalyzer(bt.analyzers.TotalValue, _name='_TotalValue')
#cerebro.addanalyzer(bt.analyzers.PositionsValue, _name='_TotalValue')
#cerebro.addobserver(bt.observers.CashValue)

cerebro.run()

print('Close Portfolio Value: %.2f' % cerebro.broker.getvalue())

#run_cerebro_plot(cerebro)
#b = Bokeh(style='bar', plot_mode='single', scheme=Tradimo())
#cerebro.plot(b)

#cerebro.plot()
#matplotlib inline

Starting Portfolio Value: 1000000.00
2019-12-17, BUY EXECUTED ,Price: 367.00, Cost: 920803.00, Comm 1841.61
2019-12-18, SELL EXECUTED, Price: 378.80, Cost: 22754.00, Comm 46.97
2020-01-13, SELL EXECUTED, Price: 398.80, Cost: 5872.00, Comm 12.76
2020-02-06, BUY EXECUTED ,Price: 396.80, Cost: 1190.40, Comm 2.38
2020-02-11, SELL EXECUTED, Price: 403.80, Cost: 1468.15, Comm 3.23
2020-02-13, SELL EXECUTED, Price: 414.80, Cost: 2202.22, Comm 4.98
2020-03-06, BUY EXECUTED ,Price: 396.80, Cost: 793.60, Comm 1.59
2020-03-10, BUY EXECUTED ,Price: 380.80, Cost: 6854.40, Comm 13.71
2020-03-11, SELL EXECUTED, Price: 381.80, Cost: 1101.49, Comm 2.29
2020-03-13, BUY EXECUTED ,Price: 334.80, Cost: 4352.40, Comm 8.70
2020-03-18, BUY EXECUTED ,Price: 341.80, Cost: 3418.00, Comm 6.84
2020-03-20, BUY EXECUTED ,Price: 353.80, Cost: 1769.00, Comm 3.54
2020-07-07, SELL EXECUTED, Price: 519.00, Cost: 30816.43, Comm 87.19
2020-07-09, SELL EXECUTED, Price: 545.00, Cost: 3668.62, Comm 10.90
2020-07-10, SELL EXEC

In [14]:
import backtrader.analyzers as btay


In [15]:
def create_table(df, max_rows=18):
         
        """基于dataframe，设置表格格式"""
         
        table = html.Table(
             # Header
             [
                 html.Tr(
                     [
                         html.Th(col) for col in df.columns
                     ]
                 )
             ] +
             # Body
             [
                 html.Tr(
                     [
                         html.Td(
                             df.iloc[i][col]
                         ) for col in df.columns
                     ]
                 ) for i in range(min(len(df), max_rows))
             ]   
         )
         
        return table
def run_cerebro_plot(cerebro,strategy_name="动量策略",author="yunjinqi",score = 90):
       begin_time=time.time()
       cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
       cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
       cerebro.addanalyzer(bt.analyzers.Calmar, _name='_Calmar')
       cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
       # cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name='_TimeDrawDown')
       cerebro.addanalyzer(bt.analyzers.GrossLeverage, _name='_GrossLeverage')
       cerebro.addanalyzer(bt.analyzers.PositionsValue, _name='_PositionsValue')
       cerebro.addanalyzer(bt.analyzers.LogReturnsRolling, _name='_LogReturnsRolling')
       cerebro.addanalyzer(bt.analyzers.PeriodStats, _name='_PeriodStats')
       cerebro.addanalyzer(bt.analyzers.Returns, _name='_Returns')
       cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio')
       # cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='_SharpeRatio_A')
       cerebro.addanalyzer(bt.analyzers.SQN, _name='_SQN')
       cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
       cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='_TradeAnalyzer')
       cerebro.addanalyzer(bt.analyzers.Transactions, _name='_Transactions')
       cerebro.addanalyzer(bt.analyzers.VWR, _name='_VWR')
       #cerebro.addanalyzer(bt.analyzers.TotalValue, _name='_TotalValue')
       #cerebro.addanalyzer(bt.analyzers.PositionsValue, _name='_TotalValue')
     
       results = cerebro.run()
       #plot_results(results,"/home/chanche/test.html")
       end_time=time.time()
       print("backtest consume time :{}".format(end_time-begin_time))
       performance_dict=OrderedDict()
       calmar_ratio=list(results[0].analyzers._Calmar.get_analysis().values())[-1]
       # print(calmar_ratio)
       drawdown_info=results[0].analyzers._DrawDown.get_analysis()
       average_drawdown_len=drawdown_info['len']
       average_drawdown_rate=drawdown_info['drawdown']
       average_drawdown_money=drawdown_info['moneydown']
       max_drawdown_len=drawdown_info['max']['len']
       max_drawdown_rate=drawdown_info['max']['drawdown']
       max_drawdown_money=drawdown_info['max']['moneydown']
       PeriodStats_info=results[0].analyzers._PeriodStats.get_analysis()
       average_rate=PeriodStats_info['average']
       stddev_rate=PeriodStats_info['stddev']
       positive_year=PeriodStats_info['positive']
       negative_year=PeriodStats_info['negative']
       nochange_year=PeriodStats_info['nochange']
       best_year=PeriodStats_info['best']
       worst_year=PeriodStats_info['worst']
       SQN_info=results[0].analyzers._SQN.get_analysis()
       sqn_ratio=SQN_info['sqn']
       VWR_info=results[0].analyzers._VWR.get_analysis()
       vwr_ratio=VWR_info['vwr']
       sharpe_info=results[0].analyzers._SharpeRatio.get_analysis()
       sharpe_ratio=sharpe_info['sharperatio']
       # sharpe_info=results[0].analyzers._SharpeRatio_A.get_analysis()
       performance_dict['calmar_ratio']=calmar_ratio
       performance_dict['average_drawdown_len']=average_drawdown_len
       performance_dict['average_drawdown_rate']=average_drawdown_rate
       performance_dict['average_drawdown_money']=average_drawdown_money
       performance_dict['max_drawdown_len']=max_drawdown_len
       performance_dict['max_drawdown_rate']=max_drawdown_rate
       performance_dict['max_drawdown_money']=max_drawdown_money
       performance_dict['average_rate']=average_rate
       performance_dict['stddev_rate']=stddev_rate
       performance_dict['positive_year']=positive_year
       performance_dict['negative_year']=negative_year
       performance_dict['nochange_year']=nochange_year
       performance_dict['best_year']=best_year
       performance_dict['worst_year']=worst_year
       performance_dict['sqn_ratio']=sqn_ratio
       performance_dict['vwr_ratio']=vwr_ratio
       performance_dict['sharpe_info']=sharpe_ratio
       performance_dict['omega']=0
       
       trade_dict_1=OrderedDict()
       trade_dict_2=OrderedDict()
       trade_info=results[0].analyzers._TradeAnalyzer.get_analysis()
       total_trade_num=trade_info['total']['total']
       total_trade_opened=trade_info['total']['open']
       total_trade_closed=trade_info['total']['closed']
       total_trade_len=trade_info['len']['total']
       long_trade_len=trade_info['len']['long']['total']
       short_trade_len=trade_info['len']['short']['total']
       
       longest_win_num=trade_info['streak']['won']['longest']
       longest_lost_num=trade_info['streak']['lost']['longest']
       net_total_pnl=trade_info['pnl']['net']['total']
       net_average_pnl=trade_info['pnl']['net']['average']
       win_num=trade_info['won']['total']
       win_total_pnl=trade_info['won']['pnl']['total']
       win_average_pnl=trade_info['won']['pnl']['average']
       win_max_pnl=trade_info['won']['pnl']['max']
       lost_num=trade_info['lost']['total']
       lost_total_pnl=trade_info['lost']['pnl']['total']
       lost_average_pnl=trade_info['lost']['pnl']['average']
       lost_max_pnl=trade_info['lost']['pnl']['max']
       
       trade_dict_1['total_trade_num']=total_trade_num
       trade_dict_1['total_trade_opened']=total_trade_opened
       trade_dict_1['total_trade_closed']=total_trade_closed
       trade_dict_1['total_trade_len']=total_trade_len
       trade_dict_1['long_trade_len']=long_trade_len
       trade_dict_1['short_trade_len']=short_trade_len
       trade_dict_1['longest_win_num']=longest_win_num
       trade_dict_1['longest_lost_num']=longest_lost_num
       trade_dict_1['net_total_pnl']=net_total_pnl
       trade_dict_1['net_average_pnl']=net_average_pnl
       trade_dict_1['win_num']=win_num
       trade_dict_1['win_total_pnl']=win_total_pnl
       trade_dict_1['win_average_pnl']=win_average_pnl
       trade_dict_1['win_max_pnl']=win_max_pnl
       trade_dict_1['lost_num']=lost_num
       trade_dict_1['lost_total_pnl']=lost_total_pnl
       trade_dict_1['lost_average_pnl']=lost_average_pnl
       trade_dict_1['lost_max_pnl']=lost_max_pnl
       
       
       long_num=trade_info['long']['total']
       long_win_num=trade_info['long']['won']
       long_lost_num=trade_info['long']['lost']
       long_total_pnl=trade_info['long']['pnl']['total']
       long_average_pnl=trade_info['long']['pnl']['average']
       long_win_total_pnl=trade_info['long']['pnl']['won']['total']
       long_win_max_pnl=trade_info['long']['pnl']['won']['max']
       long_lost_total_pnl=trade_info['long']['pnl']['lost']['total']
       long_lost_max_pnl=trade_info['long']['pnl']['lost']['max']
       
       short_num=trade_info['short']['total']
       short_win_num=trade_info['short']['won']
       short_lost_num=trade_info['short']['lost']
       short_total_pnl=trade_info['short']['pnl']['total']
       short_average_pnl=trade_info['short']['pnl']['average']
       short_win_total_pnl=trade_info['short']['pnl']['won']['total']
       short_win_max_pnl=trade_info['short']['pnl']['won']['max']
       short_lost_total_pnl=trade_info['short']['pnl']['lost']['total']
       short_lost_max_pnl=trade_info['short']['pnl']['lost']['max']
       
       
       trade_dict_2['long_num']=long_num
       trade_dict_2['long_win_num']=long_win_num
       trade_dict_2['long_lost_num']=long_lost_num
       trade_dict_2['long_total_pnl']=long_total_pnl
       trade_dict_2['long_average_pnl']=long_average_pnl
       trade_dict_2['long_win_total_pnl']=long_win_total_pnl
       trade_dict_2['long_win_max_pnl']=long_win_max_pnl
       trade_dict_2['long_lost_total_pnl']=long_lost_total_pnl
       trade_dict_2['long_lost_max_pnl']=long_lost_max_pnl
       trade_dict_2['short_num']=short_num
       trade_dict_2['short_win_num']=short_win_num
       trade_dict_2['short_lost_num']=short_lost_num
       trade_dict_2['short_total_pnl']=short_total_pnl
       trade_dict_2['short_average_pnl']=short_average_pnl
       trade_dict_2['short_win_total_pnl']=short_win_total_pnl
       trade_dict_2['short_win_max_pnl']=short_win_max_pnl
       trade_dict_2['short_lost_total_pnl']=short_lost_total_pnl
       trade_dict_2['short_lost_max_pnl']=short_lost_max_pnl
       
       
       len(performance_dict)==len(trade_dict_2)==len(trade_dict_1)
       df00=pd.DataFrame(index=range(18))
       df01=pd.DataFrame([performance_dict]).T
       df01.columns=['绩效指标值']
       df02=pd.DataFrame([trade_dict_1]).T
       df02.columns=['普通交易指标值']
       df03=pd.DataFrame([trade_dict_2]).T
       df03.columns=['多空交易指标值']
       df00['绩效指标']=df01.index
       df00['绩效指标值']=[round(float(i),4) for i in list(df01['绩效指标值'])]
       df00['普通交易指标']=df02.index
       df00['普通交易指标值']=[round(float(i),4) for i in list(df02['普通交易指标值'])]
       df00['多空交易指标']=df03.index
       df00['多空交易指标值']=[round(float(i),4) for i in list(df03['多空交易指标值'])]
       
       test_time=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
       # 账户价值
       df0=df1=pd.DataFrame([results[0].analyzers._TotalValue.get_analysis()]).T
       df0.columns=['total_value']
       
       # 总的杠杆
       df1=pd.DataFrame([results[0].analyzers._GrossLeverage.get_analysis()]).T
       df1.columns=['GrossLeverage']
       
       
       # 滚动的对数收益率
       df2=pd.DataFrame([results[0].analyzers._LogReturnsRolling.get_analysis()]).T
       df2.columns=['log_return']
       
       # year_rate
       df3=pd.DataFrame([results[0].analyzers._AnnualReturn.get_analysis()]).T
       df3.columns=['year_rate']
       
       # 总的持仓价值
       df4=pd.DataFrame(results[0].analyzers._PositionsValue.get_analysis()).T
       df4['total_position_value']=df4.sum(axis=1)
       
       # 定义表格组件
       
       
       app = dash.Dash()
       # app = JupyterDash('策略评估结果')
       # server = app.server
       colors = dict(background = 'white', text = 'black')
       
       app.layout = html.Div(
           style = dict(backgroundColor = colors['background']),
           children = [
               html.H1(
                   children='{}的策略评估结果'.format(strategy_name),
                   style = dict(textAlign='center', color = colors['text'])),
               html.Div(
                   children=f'策略作者 ： {author} ___ 测试时间： {test_time} ___ 测试分数为 : {score}',
                   style = dict(textAlign = 'center', color = colors['text'])),
           
               dcc.Graph(
                   id='账户价值',
                   figure = dict(
                       data = [{'x': list(df0.index), 'y': list(df0.total_value),
                                #'text':[int(i*1000)/10 for i in list(df3.year_rate)],
                                'type': 'scatter', 'name': '账户价值',
                               'textposition':"outside"}],
                       layout = dict(
                           title='账户价值',
                           plot_bgcolor = colors['background'], 
                           paper_bgcolor = colors['background'],
                           font = dict(color = colors['text'],
                          )
                       )
                   )
               ),
               
               dcc.Graph(
                   id='持仓市值',
                   figure = dict(
                       data = [{'x': list(df4.index), 'y': list(df4.total_position_value),
                                #'text':[int(i*1000)/10 for i in list(df3.year_rate)],
                                'type': 'scatter', 'name': '持仓市值',
                               'textposition':"outside"}],
                       layout = dict(
                           title='持仓市值',
                           plot_bgcolor = colors['background'], 
                           paper_bgcolor = colors['background'],
                           font = dict(color = colors['text']),
                       )
                   )
               ),
               dcc.Graph(
                   id='年化收益',
                   figure = dict(
                       data = [{'x': list(df3.index), 'y': list(df3.year_rate),
                                'text':[int(i*1000)/10 for i in list(df3.year_rate)],
                                'type': 'bar', 'name': '年收益率',
                               'textposition':"outside"}],
                       layout = dict(
                           title='年化收益率',
                           plot_bgcolor = colors['background'], 
                           paper_bgcolor = colors['background'],
                           font = dict(color = colors['text']),
                       )
                   )
               ),
               create_table(df00)
                               
               
           ]
       )
     
       app.run_server(debug=True, host='0.0.0.0')
       return results

In [1]:
import scipy

In [4]:
    def t_get_data_futu(code):

        quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
        ret, stock_data, page_req_key = quote_ctx.request_history_kline(code, start='2018-09-01', end='2019-09-01')  
        quote_ctx.close() 
        print(stock_data)
        stock_data['date']=pd.to_datetime(stock_data['time_key'])

        stock_data=stock_data.set_index('date',drop=True)
        stock_data=stock_data.drop(columns=['time_key'])
        
        stock_data['openinterest']=0
        
        return stock_data

In [ ]:
t_get_data_futu